# Aula 20 – Classificadores Bayesianos

## 1. Introdução

Como começamos a discutir na aula passada, o processo de identificação de uma imagem trata de atribuir uma classificação para o “dado” imagem após pré-processamento e redução de dimensionalidade. O algoritmo K-NN atribui um dado a uma classe pela semelhança (proximidade) entre o dado a classificar e dados pré-classificados. Esse método, entretanto, não dá a certeza de que a classificação será correta nem informa sua probabilidade de estar correto.

Métodos estatísticos classificam o dado informando as probabilidades de classes para os dados. Já que dados de uma mesma classe não são necessariamente iguais, ter o nível de incerteza para a classificação é importante. A maioria dos métodos de visão computacional, portanto, trabalham exatamente dessa forma, calculando estatisticamente a classificação de um dado.

No caso dos classificadores Bayesianos, queremos saber a probabilidade de estarmos diante de  uma determinada classe, dada a imagem (na verdade, o dado tratado que representa a imagem). Ou seja, é o cálculo de uma probabilidade condicional seguindo o teorema de Bayes.

Se Ci é uma classe e X é o dado que representa a imagem, temos:

## P(Ci | X) = P(Ci)*P( X | Ci) / P(X),

ou seja, a probabilidade de termos a classe i (por exemplo, termos um gato) dado X (representação da imagem após pré-processamento e redução de dimensionalidade) é igual à probabilidade da classe Ci (probabilidade de um gato aparecer em qualquer imagem) vezes a probabilidade do dado X aparecer dada a classe Ci (a probabilidade daquele tipo de dado aparecer entre todas as imagens de gato) dividido pela probabilidade do dado X aparecer entre todas as imagens possíveis.

O grande problema para conseguir computar a função acima é conhecer essas probabilidades. Quantas imagens existem? Quantas imagens são de gato? Qual a probabilidade uma determinada imagem aparecer? O cálculo, portanto, é feito assumindo-se algumas condições iniciais, como, por exemplo, uma distribuição normal das imagens e os parâmetros vão sendo ajustados iterativamente, de acordo com os erros e acertos do classificador (quanto mais imagens ele for vendo, mais precisas vão sendo as estimativas das probabilidades). 

Para mais detalhes sobre a implementação do treinamento desse e outros classificadores, sugiro cursar a disciplina de Aprendizagem de Máquina. De fato, a partir de agora, a disciplina de Visão Computacional, que até então tratava da aplicação dos métodos de Processamento de Imagens, passa a tratar da aplicação dos métodos de Aprendizagem de Máquina.

## 2. Leitura Complementar

Livro E – Seção 11.4 

Livro 1 – Seção 9.2

Livro 3 – Seção 12.2.2

## 3. Exercícios

1. Baixe umas imagens de gatos e de cachorros no Google e treine um classificador Bayesiano em OpenCV (faça o treinamento com as imagens pré-processadas e com dimensionalidade reduzida, como no K-NN). A biblioteca já traz implementado o classificador. Feito isso, procure novas imagens diferentes das que você utilizou no treino e teste seu classificador. Qual taxa de acerto você obteve?

In [7]:
import cv2
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.utils import shuffle


def prepare_dataset(train_dir, val_dir, size):
  
  # training set
  
  directory = os.listdir(train_dir + '/Dog')
  train_dogs = []
  train_dogs_labels = []
  for img in directory:
    train_dogs.append(cv2.resize(cv2.imread(train_dir + '/Dog/' + img), (size, size)).reshape(-1))
    train_dogs_labels.append(1)
    
  train_dogs = np.array(train_dogs)
  train_dogs_labels = np.array(train_dogs_labels)
  directory = os.listdir(train_dir + '/Cat')
  train_cats = []
  train_cats_labels = []
  for img in directory:
    train_cats.append(cv2.resize(cv2.imread(train_dir + '/Cat/' + img), (size, size)).reshape(-1))
    train_cats_labels.append(0)

  train_cats = np.array(train_cats)
  train_cats_labels = np.array(train_cats_labels)
  
  # validation set
  
  directory = os.listdir(val_dir + '/Dog')
  val_dogs = []
  val_dogs_labels = []
  for img in directory:
    val_dogs.append(cv2.resize(cv2.imread(val_dir + '/Dog/' + img), (size, size)).reshape(-1))
    val_dogs_labels.append(1)
    
  val_dogs = np.array(val_dogs)
  val_dogs_labels = np.array(val_dogs_labels)
  
  directory = os.listdir(val_dir + '/Cat')
  val_cats = []
  val_cats_labels = []
  for img in directory:
    val_cats.append(cv2.resize(cv2.imread(val_dir + '/Cat/' + img), (size, size)).reshape(-1))
    val_cats_labels.append(0)

  val_cats = np.array(val_cats)
  val_cats_labels = np.array(val_cats_labels)
  
  
  x_train = np.concatenate((train_dogs, train_cats))
  y_train = np.concatenate((train_dogs_labels, train_cats_labels))
  
  x_test = np.concatenate((val_dogs, val_cats))
  y_test = np.concatenate((val_dogs_labels, val_cats_labels))
  
  x_train, y_train = shuffle(x_train, y_train, random_state=123)
  x_test, y_test = shuffle(x_test, y_test, random_state=123)
  
  x_train = x_train.astype('float32') / 255.
  x_test = x_test.astype('float32') / 255.
  
  pca = PCA(n_components=200)
  pca.fit(x_train)

  x_train = pca.transform(x_train)
  x_test = pca.transform(x_test)
  
  return x_train, y_train, x_test, y_test

In [8]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


dataset_path = "data"
training_set = dataset_path + "/train"
validation_set =dataset_path + "/valid"

x_train, y_train, x_test, y_test = prepare_dataset(training_set, validation_set, size=100)

gnbayes = GaussianNB()
gnbayes.fit(x_train, y_train)
score_gnbayes = gnbayes.score(x_test, y_test)
print(score_gnbayes)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.